In [4]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [5]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [8]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [9]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [10]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [12]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [13]:
df['loc']

1     (9001 Collins Ave s 607, Surfside, FL 33154, U...
2     (1000 Biscayne Blvd #1701, Miami, FL 33132, US...
3     (15701 Collins Ave #2702, North Miami Beach, F...
4     (15701 Collins Ave #2101, North Miami Beach, F...
5     (16047 Collins Ave APT 3204, Sunny Isles Beach...
6     (Asia, 900 Brickell Key Blvd APT 1704, Miami, ...
7     (2900 NE 7th Ave Unit 1507, Miami, FL 33137, U...
8     (16001 Collins Ave APT 3804, Sunny Isles Beach...
9     (8925 Collins Ave #10b, Surfside, FL 33154, US...
10    (1425 Brickell Ave #63a, Miami, FL 33131, USA,...
Name: loc, dtype: object

In [14]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [15]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Surf Club Condo\r9001 Collins Ave S-607\rSurfside,12/7/2022,101,"$21,000,000.00",3948,"$5,319.15",Nancy Batchelor PA,"Compass Florida, LLC",Mark Kurkin,Beachfront Realty Inc,Surf Club Condo 9001 Collins Ave S-607 Surfside,Surf Club Condo,9001 Collins Ave S-607 Surfside,"(9001 Collins Ave s 607, Surfside, FL 33154, U...","(25.8775274, -80.12162339999999, 0.0)",25.877527,-80.121623,0.0
2,One Thousand Museum\r1000 Biscayne Blvd 1701\r...,12/6/2022,229,"$6,250,000.00",4600,"$1,358.70",Heather McCabe,Beachfront Realty Inc,Christopher Wands,Douglas Elliman,One Thousand Museum 1000 Biscayne Blvd 1701 Miami,One Thousand Museum,1000 Biscayne Blvd 1701 Miami,"(1000 Biscayne Blvd #1701, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0
3,Ritz-Carlton Residences\r15701 Collins Ave 270...,12/7/2022,25,"$5,900,000.00",2137,"$2,760.88",Yuliya Kachko,Oceanfront Group Inc,Dana Lund,Engel & Völkers Coconut Grove,Ritz-Carlton Residences 15701 Collins Ave 2702...,Ritz-Carlton Residences,15701 Collins Ave 2702 Sunny Isles Beach,"(15701 Collins Ave #2702, North Miami Beach, F...","(25.9219798, -80.1217032, 0.0)",25.921980,-80.121703,0.0
4,Ritz-Carlton Residences\r15701 Collins Ave 210...,12/5/2022,94,"$5,600,000.00",2688,"$2,083.33",Sonia Navarro,Miami Exclusive Properties,Luis Gautier,Fortune Christie's Int'l Real Estate,Ritz-Carlton Residences 15701 Collins Ave 2101...,Ritz-Carlton Residences,15701 Collins Ave 2101 Sunny Isles Beach,"(15701 Collins Ave #2101, North Miami Beach, F...","(25.9219798, -80.1217032, 0.0)",25.921980,-80.121703,0.0
5,Turnberry Ocean Colony\r16047 Collins Ave 3204...,12/6/2022,132,"$4,900,000.00",4480,"$1,311.91",Dina Goldentayer,Douglas Elliman,Oksana Ivanenko,"Miami Matrix Realty, LLC.",Turnberry Ocean Colony 16047 Collins Ave 3204 ...,Turnberry Ocean Colony,16047 Collins Ave 3204 Sunny Isles Beach,"(16047 Collins Ave APT 3204, Sunny Isles Beach...","(25.9252726, -80.121681, 0.0)",25.925273,-80.121681,0.0
6,Asia Condo\r900 Brickell Key Blvd 1704\rMiami,12/9/2022,37,"$2,225,000.00",2631,$845.69,Cristiano Soares,Brown Harris Stevens,Jordan Millman PA,West Avenue Realty,Asia Condo 900 Brickell Key Blvd 1704 Miami,Asia Condo,900 Brickell Key Blvd 1704 Miami,"(Asia, 900 Brickell Key Blvd APT 1704, Miami, ...","(25.7690285, -80.1856987, 0.0)",25.769029,-80.185699,0.0
7,Biscayne Beach\r2900 NE 7th Ave 1507\rMiami,12/7/2022,110,"$2,200,000.00",1868,"$1,177.73",Carolina Rocha,BHHS EWM Realty,Bradley Piper,United Realty Group Inc,Biscayne Beach 2900 NE 7th Ave 1507 Miami,Biscayne Beach,2900 NE 7th Ave 1507 Miami,"(2900 NE 7th Ave Unit 1507, Miami, FL 33137, U...","(25.8048485, -80.18598399999999, 0.0)",25.804848,-80.185984,0.0
8,Trump Tower I Condominium\r16001 Collins Ave 3...,12/7/2022,133,"$2,000,000.00",2327,$859.48,Romina Grinberg,Brown Harris Stevens,Vadim Yushkevich,Alpha Realty LLC,Trump Tower I Condominium 16001 Collins Ave 38...,Trump Tower I Condominium,16001 Collins Ave 3804 Sunny Isles Beach,"(16001 Collins Ave APT 3804, Sunny Isles Beach...","(25.9244562, -80.1216846, 0.0)",25.924456,-80.121685,0.0
9,Mirage\r8925 Collins Ave 10B\rSurfside,12/5/2022,92,"$1,675,000.00",1690,$991.12,Ana Uribe,"Related ISG Realty, LLC.",Peter Conn,"Compass Florida, LLC (PB)",Mirage 8925 Collins Ave 10B Surfside,Mirage,8925 Collins Ave 10B Surfside,"(8925 Collins Ave #10b, Surfside, FL 33154, US...","(25.8753865, -80.12113630000002, 0.0)",25.875387,-80.121136,0.0
10,0Millennium Tower Residences\r1425 Brickell Av...,12/9/2022,54,"$1,520,000.00",1523,$998.03,Jonathan Mann,Coldwell Banker Realty,Myriam Hilali,BHHS EWM Realty,0Millennium Tower Residences 1425 Brickell Ave...,,0Millennium Tower Residences 1425 Brickell Ave...,"(1425 Brickell Ave #63a, Miami, FL 33131, USA,...","(25.7591323, -80.1918295, 0.0)",25.759132

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [16]:
df.at[10,'building_name']=('Millennium Tower Residences')
df.at[10,'address_only']=('1425 Brickell Ave 63A')

In [17]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [20]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 4th - December 10th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [21]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [22]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [23]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_121022
